**kt3180** - Khwaab Thareja
<br>
**rmp10015** - Rachit Pathak
<br>
**xw3795** - Xiaozhou Wen



 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [ ]:
!nvidia-smi

Sat Nov  1 11:22:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
%%capture
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
!pip install flash-attn

In [ ]:
# %%capture
#!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
#!pip install --no-deps "xformers<0.0.26" "trl<0.9.0" "peft<0.12.0" "accelerate<0.32.0" "bitsandbytes<0.44.0" "transformers<4.43.0"

## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 6144  # Controls context length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [ ]:
from collections import Counter
import random

# balance the data set to be 50/50 true/false
def balance_dataset_undersample(dataset):
    # Count labels
    label_counts = Counter(dataset['is_correct'])
    print(f"Original distribution: {label_counts}")

    # Find the minority class count
    minority_class = min(label_counts, key=label_counts.get)
    minority_count = label_counts[minority_class]

    # Separate datasets by class
    true_samples = [ex for ex in dataset if ex['is_correct'] == True]
    false_samples = [ex for ex in dataset if ex['is_correct'] == False]

    # Undersample majority class
    if len(true_samples) > len(false_samples):
        true_samples = random.sample(true_samples, minority_count)
    else:
        false_samples = random.sample(false_samples, minority_count)

    # Combine balanced datasets
    balanced_dataset = true_samples + false_samples
    random.shuffle(balanced_dataset)

    print(f"Balanced distribution: True={len(true_samples)}, False={len(false_samples)}")
    return balanced_dataset

In [ ]:
from datasets import load_dataset, Dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = Dataset.from_list(balance_dataset_undersample(shuffled_dataset.select(range(50000)))) # Use the first 50,000 for training
validation_dataset = shuffled_dataset.select(range(50000, 52000)) # Use the next 2,000 for validation

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Original distribution: Counter({False: 30033, True: 19967})
Balanced distribution: True=19967, False=19967


In [ ]:
train_dataset.features
# confirm column names

# Compute percentage of true and false
true_count = sum(train_dataset["is_correct"])
total = len(train_dataset)
print(f"True: {true_count/total*100:.2f}%, False: {(1 - true_count/total)*100:.2f}%")

True: 50.00%, False: 50.00%


In [ ]:
import re

# The instructional prompt template for training
training_prompt = """You are a teacher with great mathematician skilled in verifying mathematical solutions, including those with code. Your task is to determine if the student's solution correctly solves the math question. Below is the teacher's question and stuent's solution.
Teacher's Question:
{}.

Student's Solution:
{}.

Student's final answer:
{}.

Output 'True' if the solution is fully correct (sound reasoning, correct logic, and accurate final answer), otherwise 'False'.

Output:
{}"""

# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# Clean the text of the solution
def clean_solution_text(solution):
    # Remove HTML tags
    cleaned = re.sub(r"</?[^>]+>", "", str(solution))
    # Remove \boxed{} notation
    cleaned = cleaned.replace("\\boxed{", "").replace("}", "")
    return cleaned.strip()

# Standardize the output to be True/False
def format_truth_value(output):
    if output is True or str(output).lower() == "true":
        return "True"
    return "False"

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    texts = []

    for i in range(len(examples["question"])):
        question = str(examples["question"][i]).strip()
        solution = clean_solution_text(examples["solution"][i])
        answer = str(examples["answer"][i]).strip()
        output = format_truth_value(examples["is_correct"][i])

        # Create formatted prompt
        text = training_prompt.format(question, solution, answer, output).strip()

        # Ensure EOS token is present exactly once
        if not text.endswith(EOS_TOKEN):
            text += EOS_TOKEN

        texts.append(text)

    return {"text": texts}

# Apply the formatting function to our training dataset and validation dataset
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
formatted_validation_dataset = validation_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/39934 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# view text to make sure that it looks good
formatted_train_dataset[1]['text']
#train_dataset[0]

"You are a teacher with great mathematician skilled in verifying mathematical solutions, including those with code. Your task is to determine if the student's solution correctly solves the math question. Below is the teacher's question and stuent's solution.\nTeacher's Question:\nIn a house of 15, 3/5 of the people eat pizza. If Aviana brought pizza with 50 pieces, and each person eating pizza took 4 pieces, calculate the number of pizza pieces that remained..\n\nStudent's Solution:\nLet's solve this problem using Python's sympy library.\n\nimport sympy as sp\n\n# house has 15 people\nnum_people_in_house = 15\n# fraction of people that eat pizza is 3/5\nfraction_of_people_that_eat_pizza = 3 / 5\nnum_people_that_eat_pizza = num_people_in_house * fraction_of_people_that_eat_pizza\n\n# pizza had 50 pieces\nnum_pizza_pieces = 50\n\n# each person takes 4 pieces\nnum_pieces_taken_by_person = 4\n\n# compute the number of pieces taken by all people\nnum_pieces_taken = num_people_that_eat_pizza

## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # use a hier rank 128, allows the model to learn more complex patterns and retain more information
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 256, # A common practice is to set alpha = 2 * r
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth 2025.10.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args = TrainingArguments(
        num_train_epochs=5,
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        warmup_steps = 300, # Gradually increases the learning rate at the start of training.
        #max_steps = 60,
        learning_rate = 6e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.005, #A regularization term that penalizes large weights to prevent overfitting and improve generalization
        lr_scheduler_type = "cosine", # Adjusts the learning rate dynamically during training. linear/cosine
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/39934 [00:00<?, ? examples/s]

## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [ ]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 39,934 | Num Epochs = 5 | Total steps = 3,120
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 335,544,320 of 8,365,805,568 (4.01% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.490700
20,0.738300
30,0.694700
40,0.659100
50,0.627200
60,0.637500
70,0.649000
80,0.627200
90,0.612800
100,0.619500


TrainOutput(global_step=3120, training_loss=0.3507182001303404, metrics={'train_runtime': 30315.1542, 'train_samples_per_second': 6.586, 'train_steps_per_second': 0.103, 'total_flos': 5.64314644601104e+18, 'train_loss': 0.3507182001303404, 'epoch': 5.0})


## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, classification_report

# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a teacher with great mathematician skilled in verifying mathematical solutions, including those with code. Your task is to determine if the student's solution correctly solves the math question. Below is the teacher's question and stuent's solution.
Teacher's Question:
{}.

Student's Solution:
{}.

Student's final answer:
{}.

Output 'True' if the solution is fully correct (sound reasoning, correct logic, and accurate final answer), otherwise 'False'.

Output:
"""

def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# --- Batch Inference Loop ---
all_predictions = []
all_ground_truths = []

print(f"Running inference on {len(validation_dataset)} examples.")

# Use tqdm for a progress bar
for i in tqdm(range(0, len(validation_dataset))):

  # Select a sample from the validation set
  example = validation_dataset[i]
  question = str(example["question"]).strip()
  solution = clean_solution_text(example["solution"])
  answer = str(example["answer"]).strip()

  # Format the prompt with the validation data
  inputs = tokenizer(
  [
    inference_prompt.format(question, solution,answer)
  ], return_tensors = "pt").to("cuda")

  # Generate the model's response
  #outputs = model.generate(**inputs, max_new_tokens = 8, temperature=0.1, use_cache = True)
  outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True)
  response = tokenizer.batch_decode(outputs)

  all_predictions.append(parse_output(response[0]))
  all_ground_truths.append(example["is_correct"])

# --- Review Results ---
print(f"Total examples: {len(all_predictions)}")
print(f"Total ground truths: {len(all_ground_truths)}")

# 1. Print the first 5 examples to check
print("\n--- First 5 Predictions ---")
for i in range(5):
    print(f"\n--- Example {i} ---")
    print(f"  Question: {validation_dataset[i]['question'][:50]}...") # Truncated
    print(f"  Prediction:     {all_predictions[i]}")
    print(f"  Correct Answer: {all_ground_truths[i]}")


# Calculate and print overall accuracy
print("\n" + "---" * 10)
print("### VALIDATION METRICS ###")
print("-" * 30)

accuracy = accuracy_score(all_ground_truths, all_predictions)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")
print(classification_report(all_ground_truths, all_predictions))

Running inference on 2000 examples.


  0%|          | 0/2000 [00:00<?, ?it/s]

Total examples: 2000
Total ground truths: 2000

--- First 5 Predictions ---

--- Example 0 ---
  Question: Determine the value of the infinite product $(2^{1...
  Prediction:     False
  Correct Answer: False

--- Example 1 ---
  Question: Mitch is baking a cake and needs 3 cups of flour a...
  Prediction:     False
  Correct Answer: False

--- Example 2 ---
  Question: Dean is playing a game with calculators. The 42 pa...
  Prediction:     False
  Correct Answer: False

--- Example 3 ---
  Question: Jenny wants to read 3 books in the next 10 days. S...
  Prediction:     False
  Correct Answer: False

--- Example 4 ---
  Question: We have a standard deck of 52 cards, with 4 cards ...
  Prediction:     False
  Correct Answer: False

------------------------------
### VALIDATION METRICS ###
------------------------------
Overall Accuracy: 85.50%
              precision    recall  f1-score   support

       False       0.86      0.90      0.88      1195
        True       0.85      0.78  

## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):

    question = str(example["question"]).strip()
    solution = clean_solution_text(example["solution"])
    answer = str(example["answer"]).strip()

    # Format the prompt with the validation data
    inputs = tokenizer(
    [
      inference_prompt.format(question, solution,answer)
    ], return_tensors = "pt").to("cuda")

    # question = example["question"]
    # solution = example["solution"]

    # # Format the prompt
    # prompt = inference_prompt.format(question, str(solution))
    # inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

100%|██████████| 10000/10000 [36:16<00:00,  4.59it/s]


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Mount google drive

### Subtask:
Mount Google Drive to save the model checkpoint.


**Reasoning**:
Mount Google Drive to save the model checkpoint.



In [ ]:
from google.colab import drive
drive.mount('/content/mydrive/', force_remount=True)

Mounted at /content/mydrive/


## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [ ]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/mydrive/MyDrive/llama3_8b_math_verifier_checkpoint_20251101"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Model checkpoint and tokenizer saved to: /content/mydrive/MyDrive/llama3_8b_math_verifier_checkpoint_20251101


## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [ ]:
# Define the path where the model checkpoint was saved in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

==((====))==  Unsloth 2025.10.10: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model and tokenizer loaded from: /content/drive/MyDrive/llama3_8b_math_verifier_checkpoint


## Generate submission file

### Subtask:
Generate the submission CSV file using the loaded model.


**Reasoning**:
Generate the submission CSV file by iterating through the test dataset, generating predictions using the loaded model, and saving the results to a pandas DataFrame.



In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):

    question = str(example["question"]).strip()
    solution = clean_solution_text(example["solution"])
    answer = str(example["answer"]).strip()

    # Format the prompt with the validation data
    inputs = tokenizer(
    [
      inference_prompt.format(question, solution,answer)
    ], return_tensors = "pt").to("cuda")

    # question = example["question"]
    # solution = example["solution"]

    # # Format the prompt
    # prompt = inference_prompt.format(question, str(solution))
    # inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

100%|██████████| 10000/10000 [1:33:05<00:00,  1.79it/s]


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.
